# 실습 09. 

**from dgl.nn import SAGEConv** 를 활용하여 GraphSAGE 모델을 구현하고 학습시켜보기

In [1]:
!pip install dgl

     |████████████████████████████████| 4.2MB 5.3MB/s 


In [2]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl # Deep Graph Library 아마존에서 기여
from dgl.data import CoraGraphDataset
from sklearn.metrics import f1_score # 성능 측정용

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:
# 하이퍼파라미터 초기화
dropoutProb = 0.5
learningRate = 1e-2
numEpochs = 50
numHiddenDim = 128
numLayers = 2
weightDecay = 5e-4
aggregatorType = "gcn"

In [4]:
'''
    Cora 데이터셋은 2708개의 논문(노드), 10556개의 인용관계(엣지)로 이루어졌습니다. 
    NumFeat은 각 노드를 나타내는 특성을 말합니다. 
    Cora 데이터셋은 각 노드가 1433개의 특성을 가지고, 개개의 특성은 '1'혹은 '0'으로 나타내어지며 특정 단어의 논문 등장 여부를 나타냅니다.
    즉, 2708개의 논문에서 특정 단어 1433개를 뽑아서, 1433개의 단어의 등장 여부를 통해 각 노드를 표현합니다.
    
    노드의 라벨은 총 7개가 존재하고, 각 라벨은 논문의 주제를 나타냅니다
    [Case_Based, Genetic_Algorithms, Neural_Networks, Probabilistic_Methods, Reinforcement_Learning, Rule_Learning, Theory]

    2708개의 노드 중, 학습에는 140개의 노드를 사용하고 모델을 테스트하는 데에는 1000개를 사용합니다.
    본 실습에서는 Validation을 진행하지않습니다.

    요약하자면, 앞서 학습시킬 모델은 Cora 데이터셋의 
    [논문 내 등장 단어들, 논문들 사이의 인용관계]를 활용하여 논문의 주제를 예측하는 모델입니다.
'''

# Cora Graph Dataset 불러오기
G = CoraGraphDataset()
numClasses = G.num_classes

G = G[0]
# 노드들의 feauture & feature의 차원
features = G.ndata['feat']
inputFeatureDim = features.shape[1]

# 각 노드들의 실제 라벨
labels = G.ndata['label']

# 학습/테스트에 사용할 노드들에 대한 표시
trainMask = G.ndata['train_mask']        
testMask = G.ndata['test_mask']

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [5]:
# 모델 학습 결과를 평가할 함수
def evaluateTrain(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

def evaluateTest(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        macro_f1 = f1_score(labels, indices, average = 'macro')
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels), macro_f1

In [6]:
def train(model, lossFunction, features, labels, trainMask, optimizer, numEpochs):
    executionTime=[]
    
    for epoch in range(numEpochs):
        model.train()

        startTime = time.time()
            
        logits = model(features)                                    # 포워딩
        loss = lossFunction(logits[trainMask], labels[trainMask])   # 모델의 예측값과 실제 라벨을 비교하여 loss 값 계산

        optimizer.zero_grad()                                       
        loss.backward()
        optimizer.step()

        executionTime.append(time.time() - startTime)

        acc = evaluateTrain(model, features, labels, trainMask)

        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f}".format(epoch, np.mean(executionTime), loss.item(), acc))

def test(model, feautures, labels, testMask):
    acc, macro_f1 = evaluateTest(model, features, labels, testMask)
    print("Test Accuracy {:.4f}".format(acc))
    print("Test macro-f1 {:.4f}".format(macro_f1)) 

In [9]:
# Define GraphSage architecture
# 기존에 구현되어 있는 SAGEConv 모듈을 불러와서, SAGEConv로 이루어진 GraphSAGE 모델을 구축한다.
from dgl.nn.pytorch.conv import SAGEConv
class GraphSAGE(nn.Module):
    '''
        graph               : 학습할 그래프
        inFeatDim           : 데이터의 feature의 차원
        numHiddenDim        : 모델의 hidden 차원
        numClasses          : 예측할 라벨의 경우의 수
        numLayers           : 인풋, 아웃풋 레이어를 제외하고 중간 레이어의 갯수
        activationFunction  : 활성화 함수의 종류
        dropoutProb         : 드롭아웃 할 확률
        aggregatorType      : [mean, gcn, pool (for max), lstm]
    '''
    '''
        SAGEConv(inputFeatDim, outputFeatDim, aggregatorType, dropoutProb, activationFunction)와 같은 형식으로 모듈 생성
    '''
    def __init__(self,graph, inFeatDim, numHiddenDim, numClasses, numLayers, activationFunction, dropoutProb, aggregatorType):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.graph = graph

        # 인풋 레이어
        self.layers.append(SAGEConv(inFeatDim, numHiddenDim, aggregatorType, dropoutProb, activationFunction))
       
        # 히든 레이어
        for i in range(numLayers):
            self.layers.append(SAGEConv(numHiddenDim, numHiddenDim, aggregatorType, dropoutProb, activationFunction)) # input, output layer 차원이 같음
        
        # 출력 레이어
        self.layers.append(SAGEConv(numHiddenDim, numClasses, aggregatorType, dropoutProb, activation=None))

    def forward(self, features):
        x = features
        for layer in self.layers:
            x = layer(self.graph, x)
        return x

In [13]:
# 모델 생성
model = GraphSAGE(G, inputFeatureDim, numHiddenDim, numClasses, numLayers, F.relu, dropoutProb, aggregatorType)
print(model)
lossFunction = torch.nn.CrossEntropyLoss()

# 옵티마이저 초기화
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)

GraphSAGE(
  (layers): ModuleList(
    (0): SAGEConv(
      (feat_drop): Dropout(p=0.5, inplace=False)
      (fc_neigh): Linear(in_features=1433, out_features=128, bias=True)
    )
    (1): SAGEConv(
      (feat_drop): Dropout(p=0.5, inplace=False)
      (fc_neigh): Linear(in_features=128, out_features=128, bias=True)
    )
    (2): SAGEConv(
      (feat_drop): Dropout(p=0.5, inplace=False)
      (fc_neigh): Linear(in_features=128, out_features=128, bias=True)
    )
    (3): SAGEConv(
      (feat_drop): Dropout(p=0.5, inplace=False)
      (fc_neigh): Linear(in_features=128, out_features=7, bias=True)
    )
  )
)


In [11]:
train(model, lossFunction, features, labels, trainMask, optimizer, numEpochs)

Epoch 00000 | Time(s) 0.4493 | Loss 1.9643 | Accuracy 0.1429
Epoch 00001 | Time(s) 0.2997 | Loss 1.9797 | Accuracy 0.6286
Epoch 00002 | Time(s) 0.2462 | Loss 1.7970 | Accuracy 0.4286
Epoch 00003 | Time(s) 0.2197 | Loss 1.6833 | Accuracy 0.8143
Epoch 00004 | Time(s) 0.2045 | Loss 1.4573 | Accuracy 0.8929
Epoch 00005 | Time(s) 0.1938 | Loss 1.1332 | Accuracy 0.9214
Epoch 00006 | Time(s) 0.1867 | Loss 0.8556 | Accuracy 0.9429
Epoch 00007 | Time(s) 0.1813 | Loss 0.5809 | Accuracy 0.9357
Epoch 00008 | Time(s) 0.1772 | Loss 0.4079 | Accuracy 0.9214
Epoch 00009 | Time(s) 0.1737 | Loss 0.2882 | Accuracy 0.9500
Epoch 00010 | Time(s) 0.1711 | Loss 0.2121 | Accuracy 0.9643
Epoch 00011 | Time(s) 0.1688 | Loss 0.1538 | Accuracy 0.9714
Epoch 00012 | Time(s) 0.1668 | Loss 0.1391 | Accuracy 0.9857
Epoch 00013 | Time(s) 0.1650 | Loss 0.1062 | Accuracy 0.9857
Epoch 00014 | Time(s) 0.1636 | Loss 0.0837 | Accuracy 0.9857
Epoch 00015 | Time(s) 0.1622 | Loss 0.0618 | Accuracy 0.9714
Epoch 00016 | Time(s) 0.

In [12]:
test(model, features, labels, testMask)

Test Accuracy 0.7760
Test macro-f1 0.7734
